In [1]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [3]:
hc = bob.HealthCheck()
hc_key = "99c0c879-0fe1-45c5-b5a6-95f0c2b1b203"
if USE_HEALTHCHECKS:
    step_1 = hc.start(hc_key)
    print(step_1)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-06-12


In [4]:
#--- Covid-19 France data updated everyday at 19:00PM
%run "covid-19_france.ipynb"

FRANCE_INIT successfully saved in FRANCE_INIT
Dataframe FRANCE_INIT successfully save in database covid-19 in MongoDB. Time: --- 0.910222053527832 secnds ---
CPU times: user 811 ms, sys: 37.6 ms, total: 849 ms
Wall time: 1.97 s
FRANCE_DB_CONCAT successfully saved in FRANCE_DB_CONCAT
Dataframe FRANCE_DB_CONCAT successfully save in database covid-19 in MongoDB. Time: --- 0.8792746067047119 secnds ---
CPU times: user 761 ms, sys: 41.5 ms, total: 803 ms
Wall time: 988 ms
FRANCE_DB_CONSO successfully saved in FRANCE_DB_CONSO
Dataframe FRANCE_DB_CONSO successfully save in database covid-19 in MongoDB. Time: --- 32.30619525909424 secnds ---
CPU times: user 15 s, sys: 2.04 s, total: 17.1 s
Wall time: 32.6 s
FRANCE_DB_ALL successfully saved in FRANCE_DB_ALL
Dataframe FRANCE_DB_ALL successfully save in database covid-19 in MongoDB. Time: --- 32.09176731109619 secnds ---
CPU times: user 17 s, sys: 2.01 s, total: 19 s
Wall time: 33.5 s
FRANCE_DB_TREND successfully saved in FRANCE_DB_TREND
Datafram

In [5]:
%%time

#--- Load data source
db_all_fr = get_datasource('FRANCE_DB_ALL')
db_trend_fr = get_datasource('FRANCE_DB_TREND')

CPU times: user 252 ms, sys: 70.7 ms, total: 323 ms
Wall time: 330 ms


In [6]:
%%time

#-- Covid-19 - DATA.GOUV.FR TENDANCES
def domain_105(df,domain_num):
    #-- Create Datamodel
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'GROUPS': domain['STATUS_NAME'],
        'GROUPS_ORDER': domain['STATUS_ORDER'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'BOTTOM_FILTER': 'Depuis le début',
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': domain['LABEL'],
        'LABEL_GROUPS': domain['LABEL_GROUPS'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
    })
    
    #-- Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARV','UPPER_FILTER_M'] = 'VARIATION EN NB'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARP','UPPER_FILTER_M'] = 'VARIATION EN %'
    
    #-- Create new date scenario in BOTTOM_FILTER
    df1 = get_lastdays(domain, 15,"15 derniers jours",'BOTTOM_FILTER')
    domain = pd.concat([domain,df1],axis=0)
    
    #-- Filter upperfilter value to get France and detailed by region
    domain_dep = domain[domain['LABEL_GROUPS'] != 'France']
    domain_fra = domain[domain['UPPER_FILTER_R'] == 'France']
    domain = pd.concat([domain_dep,domain_fra],axis=0)
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    df_save(domain,domain_num,'csv')
    if USE_MONGO:
        bob.mongo.save_df(domain,domain_num,DB_APP,True)
    
    #-- Create upper filter domain
    upper_filter = domain[['UPPER_FILTER_R']].drop_duplicates().sort_values(by='UPPER_FILTER_R', ascending=True).reset_index(drop=True)
    upper_filter['ORDER'] = upper_filter.index + 1
    upper_filter.loc[upper_filter['UPPER_FILTER_R'] == 'France', 'ORDER'] = 0
    upper_filter = upper_filter.sort_values(by='ORDER',ascending=True)
    df_save(upper_filter,'105_UPPER_FILTER','csv')
    if USE_MONGO:
        bob.mongo.save_df(upper_filter,'105_UPPER_FILTER',DB_APP,True)
    return domain

domain105 = domain_105(db_trend_fr,'105')
# domain105

105 successfully saved in 105
Dataframe 105 successfully save in database app-wsr in MongoDB. Time: --- 21.107834815979004 secnds ---
105_UPPER_FILTER successfully saved in 105_UPPER_FILTER
Dataframe 105_UPPER_FILTER successfully save in database app-wsr in MongoDB. Time: --- 0.024575233459472656 secnds ---
CPU times: user 13.6 s, sys: 1.19 s, total: 14.8 s
Wall time: 22.5 s


In [7]:
%%time

#-- Covid-19 - DATA.GOUV.FR CLASSEMENT
def domain_107(df,domain_num):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'CHILD': domain['LABEL'],
        'PARENT': domain['LABEL_GROUPS'],
        'UPPER_FILTER_M': domain['STATUS_NAME'],
        'ORDER':domain['STATUS_ORDER'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VARP': ' %',
        'PRECISION_VALUE': ',.0f',
        'PRECISION_VAR': '+,.0f',
        'PRECISION_VARP': '+,.2f',
    })
    
    domain.loc[domain['PARENT'] == 'France', 'PARENT'] = 'root'
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    df_save(domain,domain_num,'csv')
    if USE_MONGO:
        bob.mongo.save_df(domain,domain_num,DB_APP,True)
    return domain

domain107 = domain_107(db_all_fr,'107')
# domain107

107 successfully saved in 107
Dataframe 107 successfully save in database app-wsr in MongoDB. Time: --- 18.981517553329468 secnds ---
CPU times: user 9.69 s, sys: 1.08 s, total: 10.8 s
Wall time: 19.3 s


In [8]:
if USE_HEALTHCHECKS:
    step_2 = hc.done(hc_key)
    print(step_2)

Done ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-06-12
